# Logistic Regression for Sentiment Analysis

Adapted from http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

In this section, we will train a simple logistic regression model to classify movie reviews from the 50k IMDb review dataset that has been collected by Maas et. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

The dataset consists of 50,000 movie reviews from the original "train" and "test" subdirectories. The class labels are binary (1=positive and 0=negative) and contain 25,000 positive and 25,000 negative movie reviews, respectively.
For simplicity, I assembled the reviews in a single CSV file.


In [ ]:
import pandas as pd
# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

## Preprocessing Text Data

Now, let us define a simple `tokenizer` that splits the text into individual word tokens. Furthermore, we will use some simple regular expression to remove HTML markup and all non-letter characters but "emoticons," convert the text to lower case, remove stopwords, and apply the Porter stemming algorithm to convert the words into their root form.

In [1]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

Let's give it at try:

In [ ]:
tokenizer('This :) is a <a> test! :-)</br>')

## Learning (SciKit)

First, we define a generator that returns the document body and the corresponding class label:

In [2]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

To conform that the `stream_docs` function fetches the documents as intended, let us execute the following code snippet before we implement the `get_minibatch` function:

In [ ]:
_sdocs = stream_docs( path = 'shuffled_movie_data.csv' )
#print( next( _sdocs ) )
#print( next( _sdocs ) )

After we confirmed that our `stream_docs` functions works, we will now implement a `get_minibatch` function to fetch a specified number (`size`) of documents:

In [3]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

Next, we will make use of the "hashing trick" through scikit-learns [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) to create a bag-of-words model of our documents. Details of the bag-of-words model for document classification can be found at  [Naive Bayes and Text Classification I - Introduction and Theory](http://arxiv.org/abs/1410.5329).

In [4]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**20,
                         preprocessor=None, 
                         tokenizer=tokenizer)

Excercise 1: define new features according this https://web.stanford.edu/~jurafsky/slp3/7.pdf

In [6]:

## Fix a bug ( not returning tokenized by stemmer) and filtrating only nouns and adjectives
import nltk.tag

def tokenizer_modified(text) :
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    # stemming
    stemmed = [porter.stem(w) for w in text]
    stemmed_tagged = nltk.tag.pos_tag( stemmed )
    # filter by type == adjective or noun
    tokenized = [ w for (w,c) in stemmed_tagged if c == 'JJ' or c == 'NN' ]
        
    return text


vect_modified = HashingVectorizer( decode_error='ignore', 
                                   n_features=2**20,
                                   preprocessor=None, 
                                   tokenizer = tokenizer_modified )


Using the [SGDClassifier]() from scikit-learn, we will can instanciate a logistic regression classifier that learns from the documents incrementally using stochastic gradient descent. 

In [43]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='shuffled_movie_data.csv')        

Excercise 2: implement a MaxEnt classifier, using regularization, according this https://web.stanford.edu/~jurafsky/slp3/7.pdf

L2 Regularization -> penalize using the square sums of the weights by a lambda-regularization parameter

$$J_{regularized} = \small \underbrace{-\frac{1}{m} \sum\limits_{i = 1}^{m} \large{(}\small y^{(i)}\log\left(h^{(i)}\right) + (1-y^{(i)})\log\left(1- h^{(i)}\right) \large{)} }_\text{J} + \underbrace{\frac{1}{m} \frac{\lambda}{2} \sum\limits_j \theta_{j}^{2} }_\text{L2 regularization cost}$$

In [51]:

# include some sparse utils
from scipy.sparse.linalg import norm as snorm
from scipy.sparse import hstack
# pyprind as well, to check how far we are in the computations
import pyprind

class LLogisticClassifier( object ) :
    
    ### Constructor :
    ### @param: float - alpha -> learning rate
    ### @param: boolean - useRegularization -> to wheter or not use regularization
    ### @param: float - lam -> regularization parameter
    ### @param: int - max_iter -> max number of iterations to run
    def __init__( self, alpha = 0.01, useRegularization = False, lam = 0.1, max_iter = 45, batch = False ) :
        
        self.m_useRegularization = useRegularization
        self.m_lam = lam
        self.m_max_iter = max_iter
        self.m_alpha = alpha
        self.m_theta = None
        
        self.m_trained = False
        
        self.m_pbar = None
        
        ## It seems that batch gradients descent can't be used, as the memory explodes :(
        self.m_isBatch = False#batch
    
    ### @brief: Calculates the sigmoid function of a vector ( in dense or sparse form )
    ### @param: 'vector' - v -> a vector of values
    def _sigmoid( self, v ) :
        return ( 1. / ( 1. + np.exp( -v ) ) )
    
    ### @brief: Returns the theta params and the bias of the trained model
    def getModelParams( self ) :
        if not self.m_trained :
            print( 'warning, the model hasnt been trained' )
        return self.m_theta
    
    ### @brief: Trains the model given training data in matrix form
    ### @param: scipy.sparse.csr_matrix - X -> sparse matrix representation of the input data
    ### @param: list - Y -> list of output labels {0, 1}
    def train( self, X, Y ) :
        _m = X.shape[0]
        _n = X.shape[1]
        
        self.m_pbar = pyprind.ProgBar( self.m_max_iter )
        
        #if self.m_isBatch :
        if False : # Don't use batch variant, as it kind of explodes in memory
            print( 'using batch gradient descent' )
            self._trainBatch( X, Y )
        else :
            print( 'using stochastic gradient descent' )
            self._trainStochastic( X, Y )
        
    ### @brief: Trains the model given training data in matrix form
    ### @param: scipy.sparse.csr_matrix - X -> sparse matrix representation of the input data
    ### @param: list - Y -> list of output labels {0, 1}
    def partial_train( self, X, Y, passes = 1 ) :
        _m = X.shape[0]
        _n = X.shape[1]
        
        if self.m_pbar is None :
            self.m_pbar = pyprind.ProgBar( self.m_max_iter * passes )
        
        if not self.m_trained :
            self.m_theta = None
            
        self._trainStochastic( X, Y )
        
    def progBar( self ) :
        return self.m_pbar
        
    def _trainStochastic( self, X, Y ) :
        
        # m -> X.rows, n -> X.cols
        _m = X.shape[0]
        _n = X.shape[1]
        
        _col = np.ones( ( _m, 1 ) )
        X = hstack( [_col, X], "csr" )
        
        _theta = np.random.rand( _n + 1, 1 ) / 10.0
        
        if self.m_theta is not None :
            _theta = self.m_theta
        
        for q in range( self.m_max_iter ) :
            
            for j in range( _m ) :
                ## take the j training example
                _x = X[j]
                _y = Y[j]
                ## calculate the estimation using the sparse vectors
                _z = _x.dot( _theta )
                _h = self._sigmoid( _z )
                ## compute the gradient
                _e = (_h - _y)[0,0]
                _grad = _e * ( _x.transpose() )
                # update the model parameters
                if self.m_useRegularization :
                    _theta = ( 1 - ( self.m_lam / _m ) * self.m_alpha ) * _theta - self.m_alpha * _grad
                else :
                    _theta = _theta - self.m_alpha * _grad
            
            self.m_pbar.update()
            
        self.m_theta = _theta
        self.m_trained = True
    
    def _trainBatch( self, X, Y ) :
        
        # m -> X.rows, n -> X.cols
        _m = X.shape[0]
        _n = X.shape[1]
        
        _col = np.ones( ( _m, 1 ) )
        X = hstack( [_col, X], "csr" )
        
        _theta = np.random.rand( _n + 1, 1 ) / 10.0
        
        for q in range( self.m_max_iter ) :
            
            _grad = ( 1 / _m ) * ( X.transpose() ) * ( self._sigmoid( X.dot( _theta ) ) - Y )
            _theta = _theta - self.m_alpha * _grad
            
            self.m_pbar.update()
            
        self.m_theta = _theta
        self.m_trained = True
        
    def predict( self, x ) :
        if not self.m_trained :
            print( 'model hasnt been trained!' )
            return 0.0        
        
        _m = x.shape[0]
        _n = x.shape[1]

        _col = np.ones( ( _m, 1 ) )
        x = hstack( [_col, x], "csr" )
        
        _z = x.dot( self.m_theta )
        _h = self._sigmoid( _z )
            
        _yPred = 0
        if ( _h[0,0] > 0.5 ) :
            _yPred = 1
        
        return _yPred
        
    ### @brief: tests the trained model given some test data
    ### @param: scipy.sparse.csr_matrix - X -> test data in sparse matrix form
    ### @param: list - Y -> test data labels in a list
    def score( self, X, Y ) :
        if not self.m_trained :
            print( 'model hasnt been trained!' )
            return 0.0
        
        _m = X.shape[0]
        _n = X.shape[1]

        _col = np.ones( ( _m, 1 ) )
        X = hstack( [_col, X], "csr" )
        
        _correctCount = 0
        
        for j in range( _m ) :
            _x = X[j]
            _y = Y[j]
            _z = _x.dot( self.m_theta )
            _h = self._sigmoid( _z )
            
            _yPred = 0
            if ( _h[0,0] > 0.5 ) :
                _yPred = 1
            
            if _yPred == _y :
                _correctCount += 1
        
        return _correctCount / float( _m )
    
    def getTheta( self ) :
        return self.m_theta
    
    def setTheta( self, theta ) :
        self.m_trained = True
        self.m_theta = theta
    
    @staticmethod
    def saveModel( pClassifier ) :
        import joblib
        import os

        joblib.dump( pClassifier.getTheta(), './clf_modified.pkl')
        
    @staticmethod
    def loadModel() :        
        import joblib
        
        _theta = joblib.load('./clf_modified.pkl')
        
        _classifier = LLogisticClassifier()
        _classifier.setTheta( _theta )

        return _classifier


In [56]:
## Testing our classifier

USE_PRE_TRAINED = True

_lgClassifier = None

if USE_PRE_TRAINED :
    ## Load the pretrained model
    print( 'using pre-trained model' )
    _lgClassifier = LLogisticClassifier.loadModel()
    print( 'loaded' )
    
else :
    print( 'starting' )

    _lgClassifier = LLogisticClassifier( alpha = 0.1, useRegularization = True, max_iter = 10, lam = 0.5 )

    for _ in range( 100 ) :
        X_train, y_train = get_minibatch( doc_stream, size=100 )
        X_train = vect_modified.fit_transform( X_train )
        _lgClassifier.partial_train( X_train, y_train, 100 )

    print( 'done' )
    
    print( 'Accuracy on training set: %.3f' % _lgClassifier.score( X_train, y_train ) ) ## -> accuracy on training data -> ~0.950
    
    # Save model
    LLogisticClassifier.saveModel( _lgClassifier )
    
# testing set
X_test, y_test = get_minibatch(doc_stream, size=500)
X_test = vect.transform(X_test)
print( 'Accuracy on test set: %.3f' % _lgClassifier.score( X_test, y_test ) ) ## -> accuracy on test data -> ~0.704

using pre-trained model
loaded
Accuracy on test set: 0.694


In [ ]:

pbar = pyprind.ProgBar( 45 )
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    X_train = vect.fit_transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

Depending on your machine, it will take about 2-3 minutes to stream the documents and learn the weights for the logistic regression model to classify "new" movie reviews. Executing the preceding code, we used the first 45,000 movie reviews to train the classifier, which means that we have 5,000 reviews left for testing:

In [21]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))
print('Accuracy: %.3f' % clf.score(X_train, y_train)) -> accuracy on the training data -> 0.889

Accuracy: 0.578
Accuracy: 0.950


I think that the predictive performance, an accuracy of ~87%, is quite "reasonable" given that we "only" used the default parameters and didn't do any hyperparameter optimization. 

After we estimated the model perfomance, let us use those last 5,000 test samples to update our model.

In [ ]:
clf = clf.partial_fit(X_test, y_test)

<br>
<br>

# Model Persistence

In the previous section, we successfully trained a model to predict the sentiment of a movie review. Unfortunately, if we'd close this IPython notebook at this point, we'd have to go through the whole learning process again and again if we'd want to make a prediction on "new data."

So, to reuse this model, we could use the [`pickle`](https://docs.python.org/3.5/library/pickle.html) module to "serialize a Python object structure". Or even better, we could use the [`joblib`](https://pypi.python.org/pypi/joblib) library, which handles large NumPy arrays more efficiently.

To install:
conda install -c anaconda joblib

In [ ]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './vectorizer.pkl')
joblib.dump(clf, './clf.pkl')

Using the code above, we "pickled" the `HashingVectorizer` and the `SGDClassifier` so that we can re-use those objects later. However, `pickle` and `joblib` have a known issue with `pickling` objects or functions from a `__main__` block and we'd get an `AttributeError: Can't get attribute [x] on <module '__main__'>` if we'd unpickle it later. Thus, to pickle the `tokenizer` function, we can write it to a file and import it to get the `namespace` "right".

In [ ]:
%%writefile tokenizer.py
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

In [ ]:
from tokenizer import tokenizer
joblib.dump(tokenizer, './tokenizer.pkl')

Now, let us restart this IPython notebook and check if the we can load our serialized objects:

In [ ]:
import joblib
tokenizer = joblib.load('./tokenizer.pkl')
vect = joblib.load('./vectorizer.pkl')
clf = joblib.load('./clf.pkl')

After loading the `tokenizer`, `HashingVectorizer`, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful, for example, if we'd want to embed our classifier into a web application -- a topic for another IPython notebook.

In [ ]:
example = ['I hate this movie']
print( tokenizer( example[0] ) )
X = vect.transform(example)
print( X )
clf.predict(X)

In [ ]:
example = ['I loved this movie']
print( tokenizer( example[0] ) )
X = vect.transform(example)
print( X )
clf.predict(X)